In [1]:
#Imports
import numpy as np
import pandas as pd
import random, glob, time
import matplotlib.pyplot as plt

DATA_PATH = 'data/'

In [2]:
csv_files = glob.glob(DATA_PATH+"*.csv")
csv_files

['data/readings_device.39671_from.2022-07-01_to.2022-10-02.csv',
 'data/Hauler_Response_Pick-Up_2023-01-04_1018.csv',
 'data/readings_device.39686_from.2022-04-01_to.2022-07-02.csv',
 'data/readings_device.39686_from.2021-07-16_to.2021-10-02.csv',
 'data/readings_device.24668_from.2021-10-01_to.2022-01-02.csv',
 'data/readings_device.39671_from.2022-04-01_to.2022-07-02.csv',
 'data/readings_device.39667_from.2021-07-01_to.2021-10-02.csv',
 'data/readings_device.39686_from.2022-07-01_to.2022-10-02.csv',
 'data/readings_device.39874_from.2022-04-01_to.2022-07-02.csv',
 'data/readings_device.39691_from.2021-10-01_to.2022-01-02.csv',
 'data/readings_device.39667_from.2022-01-01_to.2022-04-02.csv',
 'data/readings_device.39874_from.2022-07-01_to.2022-10-02.csv',
 'data/readings_device.39569_from.2021-10-01_to.2022-01-02.csv',
 'data/readings_device.39689_from.2022-04-01_to.2022-07-02.csv',
 'data/readings_device.24572_from.2022-04-01_to.2022-07-02.csv',
 'data/readings_device.39665_from.202